In [26]:
import os
import sys
import json
from datetime import datetime
import subprocess
import pandas as pd
import plotly.graph_objects as go
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)
import Birdeye.Basics.ATH_Breakout_Master as ATH_Breakout_Master
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [27]:
# Specify the path to the Python file you want to run
python_file_path = 'Dataset_ATH_Breakout.py'

# Run the Python file
subprocess.run(['python', python_file_path])

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 30K
Max Liquidity: 1.00Mil
Min Market Cap: 500K
Max Market Cap: 50.00Mil
Filtering New Tokens for: 2 days, 23 hours, 0 minutes back
Token Launch Liquidity Filter: 30K

--------------------------------
 
 
--------------------------------

https://dexscreener.com/solana/D51NngCMfJXctfEvFue4nkRe9aWpHoAw1JzBZ7RDZFYe - Market Cap: 46.41Mil
https://dexscreener.com/solana/9DzEXkhGdtrdp1CmWwSn5bSQ6j7m8PFDc6dWSf4YCni1 - Market Cap: 40.26Mil
https://dexscreener.com/solana/6yrFPZEkSQD46TpLEBgtbuuaghGxTkW2o2cyrp5t2rxY - Market Cap: 23.06Mil
https://dexscreener.com/solana/8GFdFMyYJgRNnR4zGrzcVvK1PdxF4bhrcuvbtv3Cpump - Market Cap: 14.57Mil
https://dexscreener.com/solana/FjTJCCQpLU4fpH58mN1bTQXiQsjJVYai3QYFjYqYpump - Market Cap: 9.57Mil
https://dexscreener.com/solana/

CompletedProcess(args=['python', 'Dataset_ATH_Breakout.py'], returncode=0)

In [28]:
# Get the most recent folder
base_path = 'Data/New_Token_Data'
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = ATH_Breakout_Master.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = ATH_Breakout_Master.import_ohlcv_data(ohlcv_datetime_folder)

Importing OHLCV data from folder: Data/New_Token_Data/2024_11_12/OHLCV_Data/2024_Nov_12_0623AM


In [67]:
def plot_ath_breakouts(imported_ohlcv_data, initial_timeframe='1H', drawdown_percent=10, 
                      max_hours_separation=100, lookback_candles=6, volume_threshold=3,
                      volume_ma_period=5):
    """
    Plot price action showing breakouts to new highs with volume analysis.
    
    Parameters:
    - imported_ohlcv_data: Dictionary of OHLCV data for different timeframes
    - initial_timeframe: Starting timeframe to display (e.g., '1H', '15m', '5m', '1m')
    - drawdown_percent: Minimum required drawdown from previous high (e.g., 10 for 10%)
    - max_hours_separation: Maximum hours allowed between highs (e.g., 100)
    - lookback_candles: Number of recent candles to check for new ATH (e.g., 6)
    - volume_threshold: Minimum volume increase multiplier for highlighting (e.g., 3 for 3x)
    - volume_ma_period: Number of periods for volume moving average calculation (e.g., 5)
    """
    
    def check_recent_ath(df, drawdown_percent, max_hours_separation, lookback_candles):
        if len(df) <= lookback_candles + 10:
            return False, None, None, None
            
        # Get data excluding recent lookback period
        historical_df = df.iloc[:-lookback_candles]
        
        # Find the highest peak before lookback period
        previous_high = historical_df['high'].max()
        previous_high_idx = historical_df['high'].idxmax()
        
        # Get recent candles
        recent_df = df.iloc[-lookback_candles:]
        recent_high = recent_df['high'].max()
        recent_high_idx = recent_df['high'].idxmax()
        
        # Check if we have a new ATH in recent candles
        has_new_ath = recent_high > previous_high
        
        if not has_new_ath:
            return False, None, None, None
            
        # Find lowest point between previous high and recent high
        between_highs_df = df.loc[previous_high_idx:recent_high_idx]
        lowest_close = between_highs_df['close'].min()
        lowest_close_idx = between_highs_df['close'].idxmin()
        
        # Calculate drawdown
        drawdown = ((previous_high - lowest_close) / previous_high) * 100
        
        # Calculate time separation
        time_diff = (recent_high_idx - previous_high_idx).total_seconds() / 3600
        
        # Check conditions
        if drawdown < drawdown_percent or time_diff > max_hours_separation:
            return False, None, None, None
            
        return True, (previous_high, previous_high_idx), (lowest_close, lowest_close_idx), (recent_high, recent_high_idx)
    
    def calculate_volume_increase(df, ma_period):
        """Calculate volume percentage increase relative to prior MA"""
        volume_ma = df['volume'].rolling(window=ma_period).mean().shift(1)
        volume_increase = df['volume'] / volume_ma
        return volume_increase, volume_ma
    
    def has_outliers(df, threshold_multiplier=15.0):
        if df.empty:
            return True
            
        median_price = df['close'].median()
        high_outliers = df['high'] > (median_price * threshold_multiplier)
        
        if not high_outliers.any():
            return False
            
        outlier_indices = df.index[high_outliers]
        for idx in outlier_indices:
            start_idx = max(0, df.index.get_loc(idx) - 3)
            end_idx = min(len(df), df.index.get_loc(idx) + 4)
            window = df.iloc[start_idx:end_idx]
            
            window_median = window['close'].median()
            if df.loc[idx, 'high'] > window_median * threshold_multiplier:
                return True
                
        return False
    
    # Define color scheme
    BACKGROUND_COLOR = '#1e1e1e'
    TEXT_COLOR = '#ffffff'
    GRID_COLOR = 'rgba(51, 51, 51, 0.3)'
    TITLE_COLOR = '#00cc00'
    PREVIOUS_HIGH_COLOR = '#FFA500'  # Neon orange
    NEW_HIGH_COLOR = '#32CD32'      # Lime green
    LOW_POINT_COLOR = '#FF8C00'     # Dark orange
    LEGEND_TEXT_COLOR = '#FFFDD0'   # Heavy cream for legend
    SEPARATOR_COLOR = '#555555'     # Darker gray for separator
    INCREASING_COLOR = '#00ff9d'    # Bright green for up moves
    DECREASING_COLOR = '#ff005b'    # Bright red for down moves
    
    available_timeframes = ['1m', '5m', '15m', '1H']
    token_data = []
    
    # Process market cap data
    base_path = 'Data/New_Token_Data'
    date_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('2024_')]
    most_recent_date = max(date_folders)
    date_folder = os.path.join(base_path, most_recent_date)
    
    summary_folder = os.path.join(date_folder, 'Token_Summary')
    datetime_folders = [f for f in os.listdir(summary_folder) if os.path.isdir(os.path.join(summary_folder, f))]
    most_recent_datetime = max(datetime_folders)
    summary_datetime_folder = os.path.join(summary_folder, most_recent_datetime)
    
    mc_files = [f for f in os.listdir(summary_datetime_folder) if f.startswith('new_tokens_mc') and f.endswith('.csv')]
    latest_mc_file = os.path.join(summary_datetime_folder, mc_files[0])
    market_caps = pd.read_csv(latest_mc_file, index_col=0)['Market Cap']
    
    # Calculate and filter tokens with new ATHs
    for token_address, timeframes in imported_ohlcv_data.items():
        if initial_timeframe not in timeframes:
            continue
            
        df_initial = timeframes[initial_timeframe]
        if df_initial.empty or has_outliers(df_initial):
            continue
            
        has_pattern, first_high_data, low_point_data, new_high_data = check_recent_ath(
            df_initial, drawdown_percent, max_hours_separation, lookback_candles)
            
        if has_pattern:
            token_data.append({
                'address': token_address,
                'first_high_data': first_high_data,
                'low_point_data': low_point_data,
                'new_high_data': new_high_data
            })
    
    # Create plots for each token
    for token_data in token_data:
        token_address = token_data['address']
        gmgn_link = f"https://gmgn.ai/sol/token/{token_address}"
        dexscreener_link = f"https://dexscreener.com/solana/{token_address}"
        print(f"\nProcessing: {gmgn_link}\n{dexscreener_link}")
        
        timeframes = imported_ohlcv_data[token_address]
        market_cap = ATH_Breakout_Master.format_number(market_caps.get(token_address, 0))
        
        # Create subplots with shared x-axis and secondary y-axis for volume
        fig = make_subplots(rows=2, cols=1, 
                           shared_xaxes=True,
                           vertical_spacing=0.08,
                           row_heights=[0.7, 0.3],
                           specs=[[{"type": "candlestick"}],
                                 [{"secondary_y": True}]])
        
        timeframe_volumes = {}
        
        for tf in available_timeframes:
            if tf not in timeframes:
                continue
                
            df = timeframes[tf].copy()
            if df.empty:
                continue
                
            timeframe_volumes[tf] = df['volume'].iloc[-1]
            
            # Add candlestick chart
            fig.add_trace(
                go.Candlestick(
                    x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'],
                    name="Price",
                    showlegend=False,
                    increasing_line_color=INCREASING_COLOR,
                    decreasing_line_color=DECREASING_COLOR,
                    visible=(tf == initial_timeframe)
                ),
                row=1, col=1
            )
            
            # Add previous high triangle
            first_high, first_idx = token_data['first_high_data']
            fig.add_trace(
                go.Scatter(
                    x=[first_idx],
                    y=[first_high],
                    mode='markers+text',
                    marker=dict(
                        symbol='triangle-up',
                        size=20,
                        color=PREVIOUS_HIGH_COLOR,
                        line=dict(color=PREVIOUS_HIGH_COLOR, width=2)
                    ),
                    name='Previous High',
                    showlegend=True,
                    legendgroup="price_markers",
                    legendgrouptitle_font=dict(color=LEGEND_TEXT_COLOR),
                    visible=(tf == initial_timeframe)
                ),
                row=1, col=1
            )
            
            # Add lowest point with drawdown %
            low_point, low_idx = token_data['low_point_data']
            drawdown = ((first_high - low_point) / first_high) * 100
            fig.add_trace(
                go.Scatter(
                    x=[low_idx],
                    y=[low_point],
                    mode='markers+text',
                    marker=dict(
                        symbol='diamond',
                        size=10,
                        color=LOW_POINT_COLOR,
                        line=dict(color=LOW_POINT_COLOR, width=2)
                    ),
                    text=f"{int(drawdown)}%",
                    textposition="bottom center",
                    textfont=dict(color=TEXT_COLOR),
                    name='Lowest Close',
                    showlegend=True,
                    legendgroup="price_markers",
                    legendgrouptitle_font=dict(color=LEGEND_TEXT_COLOR),
                    visible=(tf == initial_timeframe)
                ),
                row=1, col=1
            )
            
            # Add new high triangle
            new_high, new_idx = token_data['new_high_data']
            fig.add_trace(
                go.Scatter(
                    x=[new_idx],
                    y=[new_high],
                    mode='markers+text',
                    marker=dict(
                        symbol='triangle-up',
                        size=20,
                        color=NEW_HIGH_COLOR,
                        line=dict(color=NEW_HIGH_COLOR, width=2)
                    ),
                    name='New High',
                    showlegend=True,
                    legendgroup="price_markers",
                    legendgrouptitle_font=dict(color=LEGEND_TEXT_COLOR),
                    visible=(tf == initial_timeframe)
                ),
                row=1, col=1
            )
            
            # Calculate volume colors and volume increase
            colors = [INCREASING_COLOR if df['close'].iloc[i] >= df['open'].iloc[i] 
                     else DECREASING_COLOR for i in range(len(df))]
            
            volume_increase, _ = calculate_volume_increase(df, volume_ma_period)
            
            # Add volume bars
            fig.add_trace(
                go.Bar(
                    x=df.index,
                    y=df['volume'],
                    name="Volume",
                    marker_color=colors,
                    showlegend=False,
                    visible=(tf == initial_timeframe)
                ),
                row=2, col=1,
                secondary_y=False
            )
            
            # Add volume spike markers with matching colors
            # Inside the plotting loop, replace the volume spike section:
            volume_spikes = volume_increase[volume_increase >= volume_threshold]
            if not volume_spikes.empty:
                # Calculate colors based on price change percentage
                spike_colors = []
                for timestamp in volume_spikes.index:
                    candle_idx = df.index.get_loc(timestamp)
                    open_price = df['open'].iloc[candle_idx]
                    close_price = df['close'].iloc[candle_idx]
                    
                    # Calculate percentage change
                    pct_change = abs((close_price - open_price) / open_price * 100)
                    
                    # Adjust brightness based on percentage change
                    # Higher percentage = brighter color
                    brightness = min(100 + (pct_change * 2), 255) / 255  # Caps at 255
                    
                    if close_price >= open_price:
                        # For green candles: brighten the base green color
                        r = int(0 * brightness)
                        g = int(255 * brightness)
                        b = int(157 * brightness)
                        color = f'rgb({r},{g},{b})'
                    else:
                        # For red candles: brighten the base red color
                        r = int(255 * brightness)
                        g = int(0 * brightness)
                        b = int(91 * brightness)
                        color = f'rgb({r},{g},{b})'
                        
                    spike_colors.append(color)
                
                fig.add_trace(
                    go.Scatter(
                        x=volume_spikes.index,
                        y=volume_spikes,
                        mode='markers+text',
                        marker=dict(
                            symbol='circle',
                            size=22,
                            color=spike_colors,
                            line=dict(color=spike_colors, width=2)
                        ),
                        text=[f"{int(v) if v != float('inf') else '∞'}" for v in volume_spikes],
                        textposition="middle center",
                        textfont=dict(
                            color='black',
                            size=11,
                            family="Arial Black"
                        ),
                        name=f"Volume Spikes (≥{volume_threshold}x)",
                        showlegend=True,
                        legendgroup="volume_markers",
                        legendgrouptitle_font=dict(color=LEGEND_TEXT_COLOR),
                        visible=(tf == initial_timeframe)
                    ),
                    row=2, col=1,
                    secondary_y=True
                )
        
        # Update layout
        fig.update_layout(
            height=800,
            width=1200,
            paper_bgcolor=BACKGROUND_COLOR,
            plot_bgcolor=BACKGROUND_COLOR,
            margin=dict(r=50, t=80, l=50, b=50),
            showlegend=True,
            legend=dict(
                font=dict(color=LEGEND_TEXT_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR
            ),
            hovermode='x unified',
            title=dict(
                text=f"Drawdown: {drawdown:.1f}% | Market Cap: {market_cap}",
                font=dict(size=16, color=TITLE_COLOR),
                x=0.5,
                y=0.95
            ),
            xaxis_rangeslider_visible=False
        )
        
        # Add separator line between plots
        fig.add_shape(
            type="line",
            x0=0,
            y0=0,
            x1=1,
            y1=0,
            xref="paper",
            yref="paper",
            line=dict(
                color=SEPARATOR_COLOR,
                width=2,
                dash="solid"
            )
        )
        
        # Update axes properties
        for i in range(2):
            fig.update_xaxes(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showline=True,
                linecolor=GRID_COLOR,
                tickfont=dict(color=TEXT_COLOR),
                row=i+1, col=1
            )
            
            fig.update_yaxes(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showticklabels=True,
                showline=True,
                linecolor=GRID_COLOR,
                zeroline=False,
                fixedrange=True,
                tickfont=dict(color=TEXT_COLOR),
                row=i+1, col=1
            )
        
        # Update secondary y-axis for volume spikes
        fig.update_yaxes(
            title_text="Volume Multiple",
            title_font=dict(color=TEXT_COLOR),
            tickfont=dict(color=TEXT_COLOR),
            row=2, col=1,
            secondary_y=True
        )
        
        fig.show()

# Example usage with typical values
plot_ath_breakouts(imported_ohlcv_data, 
                   initial_timeframe='15m',          # Starting timeframe
                   drawdown_percent=30,             # 10% minimum drawdown
                   max_hours_separation=24,        # 100 hours max between highs
                   lookback_candles=6,             # Check last 6 candles for new ATH
                   volume_threshold=6,             # Show volume spikes >= 3x MA
                   volume_ma_period=4)             # 4-period volume MA


Processing: https://gmgn.ai/sol/token/A6jBzMFsrxGEi2s8SDejWHojGu2j2RF7LvwTD2NTpump
https://dexscreener.com/solana/A6jBzMFsrxGEi2s8SDejWHojGu2j2RF7LvwTD2NTpump



Processing: https://gmgn.ai/sol/token/6QciU9YcYiCEbxPKWZT6UfyWGfzqiqGjhSn3F7Nvpump
https://dexscreener.com/solana/6QciU9YcYiCEbxPKWZT6UfyWGfzqiqGjhSn3F7Nvpump



Processing: https://gmgn.ai/sol/token/EL9FuDNNUyjjmQ4U8a1Aazjzq5sDz4krhvBjesAfpump
https://dexscreener.com/solana/EL9FuDNNUyjjmQ4U8a1Aazjzq5sDz4krhvBjesAfpump



Processing: https://gmgn.ai/sol/token/8467ssuj6Gkw15ABv6BvxJAKnGJALXJu6dxDxt4upump
https://dexscreener.com/solana/8467ssuj6Gkw15ABv6BvxJAKnGJALXJu6dxDxt4upump


In [40]:
def plot_ath_breakouts(imported_ohlcv_data, initial_timeframe='5m'):
    """
    Plot price action showing previous ATH and new ATH from recent candles.
    Mobile-optimized with clean display and improved visuals.
    """
    
    def check_recent_ath(df, lookback=4):
        if len(df) <= lookback:
            return False, None, None
            
        # Get previous ATH (excluding last 4 candles)
        historical_df = df.iloc[:-lookback]
        previous_ath = historical_df['high'].max()
        previous_ath_idx = historical_df['high'].idxmax()
        
        # Check last 4 candles
        recent_df = df.iloc[-lookback:]
        recent_high = recent_df['high'].max()
        recent_ath_idx = recent_df['high'].idxmax()
        
        has_new_ath = recent_high > previous_ath
        return has_new_ath, (previous_ath, previous_ath_idx), (recent_high, recent_ath_idx)
    
    def create_timeframe_annotations(timeframe, market_cap, volume):
        formatted_volume = ATH_Breakout_Master.format_number(volume)
        return [
            dict(
                text=f"<b>Timeframe:</b> {timeframe}",
                x=0.2,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Current MC:</b> {market_cap}",
                x=0.5,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Prior Candle Vol:</b> {formatted_volume}",
                x=0.8,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            )
        ]
    
    def has_outliers(df, threshold_multiplier=15.0):
        if df.empty:
            return True
            
        median_price = df['close'].median()
        high_outliers = df['high'] > (median_price * threshold_multiplier)
        
        if not high_outliers.any():
            return False
            
        outlier_indices = df.index[high_outliers]
        for idx in outlier_indices:
            start_idx = max(0, df.index.get_loc(idx) - 3)
            end_idx = min(len(df), df.index.get_loc(idx) + 4)
            window = df.iloc[start_idx:end_idx]
            
            window_median = window['close'].median()
            if df.loc[idx, 'high'] > window_median * threshold_multiplier:
                return True
                
        return False
    
    # Define color scheme with fainter grid
    BACKGROUND_COLOR = '#1e1e1e'
    TEXT_COLOR = '#ffffff'
    GRID_COLOR = 'rgba(51, 51, 51, 0.3)'
    TITLE_COLOR = '#00cc00'
    PREVIOUS_ATH_COLOR = '#FFD700'  # Gold for previous ATH
    NEW_ATH_COLOR = '#FF0000'       # Bright red for new ATH
    
    available_timeframes = ['1m', '5m', '15m', '1H']
    token_data = []
    
    # Process market cap data
    base_path = 'Data/New_Token_Data'
    date_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('2024_')]
    most_recent_date = max(date_folders)
    date_folder = os.path.join(base_path, most_recent_date)
    
    summary_folder = os.path.join(date_folder, 'Token_Summary')
    datetime_folders = [f for f in os.listdir(summary_folder) if os.path.isdir(os.path.join(summary_folder, f))]
    most_recent_datetime = max(datetime_folders)
    summary_datetime_folder = os.path.join(summary_folder, most_recent_datetime)
    
    mc_files = [f for f in os.listdir(summary_datetime_folder) if f.startswith('new_tokens_mc') and f.endswith('.csv')]
    latest_mc_file = os.path.join(summary_datetime_folder, mc_files[0])
    market_caps = pd.read_csv(latest_mc_file, index_col=0)['Market Cap']
    
    # Calculate and filter tokens with new ATHs
    for token_address, timeframes in imported_ohlcv_data.items():
        if initial_timeframe not in timeframes:
            continue
            
        df_initial = timeframes[initial_timeframe]
        if df_initial.empty or has_outliers(df_initial):
            continue
            
        has_new_ath, previous_ath_data, recent_ath_data = check_recent_ath(df_initial)
        if has_new_ath:
            token_data.append({
                'address': token_address,
                'previous_ath_data': previous_ath_data,
                'recent_ath_data': recent_ath_data
            })
    
    # Create plots for each token
    for token_data in token_data:
        token_address = token_data['address']
        gmgn_link = f"https://gmgn.ai/sol/token/{token_address}"
        dexscreener_link = f"https://dexscreener.com/solana/{token_address}"
        print(f"\nProcessing: {gmgn_link}\n{dexscreener_link}")
        
        timeframes = imported_ohlcv_data[token_address]
        market_cap = ATH_Breakout_Master.format_number(market_caps.get(token_address, 0))
        
        fig = go.Figure(layout_xaxis_rangeslider_visible=False)
        timeframe_traces = {}
        timeframe_volumes = {}
        
        # Process each timeframe
        for tf in available_timeframes:
            if tf not in timeframes:
                continue
                
            df = timeframes[tf]
            if df.empty:
                continue
                
            traces = []
            timeframe_volumes[tf] = df['volume'].iloc[-1]
            
            # Add candlestick chart
            traces.append(go.Candlestick(
                x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name="Price",
                showlegend=False,
                increasing_line_color='#00ff9d',
                decreasing_line_color='#ff005b',
                visible=(tf == initial_timeframe)
            ))
            
            # Add previous ATH star
            prev_ath, prev_idx = token_data['previous_ath_data']
            traces.append(go.Scatter(
                x=[prev_idx],
                y=[prev_ath],
                mode='markers',
                marker=dict(
                    symbol='star',
                    size=24,
                    color=PREVIOUS_ATH_COLOR,
                    line=dict(
                        color=PREVIOUS_ATH_COLOR,
                        width=2
                    )
                ),
                name='Previous ATH',
                showlegend=True,
                visible=(tf == initial_timeframe)
            ))
            
            # Add new ATH star
            recent_ath, recent_idx = token_data['recent_ath_data']
            traces.append(go.Scatter(
                x=[recent_idx],
                y=[recent_ath],
                mode='markers',
                marker=dict(
                    symbol='star',
                    size=24,
                    color=NEW_ATH_COLOR,
                    line=dict(
                        color=NEW_ATH_COLOR,
                        width=2
                    )
                ),
                name='New ATH',
                showlegend=True,
                visible=(tf == initial_timeframe)
            ))
            
            timeframe_traces[tf] = traces
        
        # Create mobile-friendly timeframe buttons
        buttons = []
        for tf in available_timeframes:
            if tf not in timeframe_traces:
                continue
                
            visible_traces = []
            for other_tf in timeframe_traces:
                visible_traces.extend([tf == other_tf] * len(timeframe_traces[other_tf]))
            
            tf_annotations = create_timeframe_annotations(
                tf,
                market_cap,
                timeframe_volumes[tf]
            )
            
            buttons.append(dict(
                label=tf,
                method="update",
                args=[
                    {"visible": visible_traces},
                    {"annotations": tf_annotations}
                ]
            ))
        
        # Add all traces to the figure
        for traces in timeframe_traces.values():
            fig.add_traces(traces)
        
        # Create initial annotations
        initial_annotations = create_timeframe_annotations(
            initial_timeframe,
            market_cap,
            timeframe_volumes[initial_timeframe]
        )
        
        # Update layout with fixed dimensions
        fig.update_layout(
            height=800,
            width=1200,
            paper_bgcolor=BACKGROUND_COLOR,
            plot_bgcolor=BACKGROUND_COLOR,
            margin=dict(r=50, t=80, l=50, b=50),
            showlegend=True,  # Changed to True to show ATH stars in legend
            hovermode='x unified',
            annotations=initial_annotations,
            updatemenus=[dict(
                type="buttons",
                direction="right",
                x=0.5,
                y=1.05,
                xanchor="center",
                yanchor="bottom",
                showactive=True,
                active=available_timeframes.index(initial_timeframe),
                buttons=buttons,
                bgcolor=BACKGROUND_COLOR,
                bordercolor="#555555",
                borderwidth=2,
                font=dict(
                    size=16,
                    color=TITLE_COLOR
                ),
                pad=dict(r=20, t=10, b=10, l=20)
            )],
            xaxis=dict(
                showticklabels=False,
                showgrid=True,
                gridcolor=GRID_COLOR,
                showline=True,
                linecolor=GRID_COLOR,
                rangeslider=dict(visible=False)
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showticklabels=False,
                showline=False,
                zeroline=False,
                fixedrange=True
            )
        )
        
        fig.show()

# Call the function
plot_ath_breakouts(imported_ohlcv_data, initial_timeframe='15m')


Processing: https://gmgn.ai/sol/token/8pDajezgfVP6yTAVAgLkosTE4PKCQyc9mpHdPiqvpump
https://dexscreener.com/solana/8pDajezgfVP6yTAVAgLkosTE4PKCQyc9mpHdPiqvpump



Processing: https://gmgn.ai/sol/token/HLH5Qn92L3L9Ebm96Sa8rygepe4WaEQGotpqgUeSG9Jd
https://dexscreener.com/solana/HLH5Qn92L3L9Ebm96Sa8rygepe4WaEQGotpqgUeSG9Jd



Processing: https://gmgn.ai/sol/token/A6jBzMFsrxGEi2s8SDejWHojGu2j2RF7LvwTD2NTpump
https://dexscreener.com/solana/A6jBzMFsrxGEi2s8SDejWHojGu2j2RF7LvwTD2NTpump



Processing: https://gmgn.ai/sol/token/6QciU9YcYiCEbxPKWZT6UfyWGfzqiqGjhSn3F7Nvpump
https://dexscreener.com/solana/6QciU9YcYiCEbxPKWZT6UfyWGfzqiqGjhSn3F7Nvpump



Processing: https://gmgn.ai/sol/token/82muMHDFVwG3c5iTxhWYw3q75G8nHyTqLpdKwMQTFwQd
https://dexscreener.com/solana/82muMHDFVwG3c5iTxhWYw3q75G8nHyTqLpdKwMQTFwQd



Processing: https://gmgn.ai/sol/token/ABgHHrBxAYC5aZUXTUS6ERxFTL8KeNjtbp1K1G3SXoQi
https://dexscreener.com/solana/ABgHHrBxAYC5aZUXTUS6ERxFTL8KeNjtbp1K1G3SXoQi



Processing: https://gmgn.ai/sol/token/8467ssuj6Gkw15ABv6BvxJAKnGJALXJu6dxDxt4upump
https://dexscreener.com/solana/8467ssuj6Gkw15ABv6BvxJAKnGJALXJu6dxDxt4upump



Processing: https://gmgn.ai/sol/token/FuQASH8ps9NPeDu4h3rVtMBygKYoSiSTZ4uSiA5tpump
https://dexscreener.com/solana/FuQASH8ps9NPeDu4h3rVtMBygKYoSiSTZ4uSiA5tpump



Processing: https://gmgn.ai/sol/token/Gaq4vWNWQiMyVtrXGHSMVMbLP2Cd8CntxQY5kh8SGTUb
https://dexscreener.com/solana/Gaq4vWNWQiMyVtrXGHSMVMbLP2Cd8CntxQY5kh8SGTUb
